# Polygenic risk scores for hypertension data

> Polygenic risk scores are important tools for understanding complex genetic associations. In this notebook, we show how to derive polygenic scores based on summary statistics and a matrix of correlation between genetic variants. We will use R package bigsnpr that implements the LDpred2 method (https://doi.org/10.1093/bioinformatics/btaa1029).

> As input, we will use the hypertension example data used before for GWAS example. This notebook focus on a logistic regression model using simulated participant data.

- runtime: <30m
- recommended instance: mem1_ssd1_v2_x16
- estimated cost: <£0.50

This notebook depends on:
* **Notebook 201** - pheno_data_hypertension.csv
* **Notebook 107** - maf_flt_8chroms* prefixed files

## Install required packages

Function `p_load` from `pacman` loads packages into R.
If a given package missing it will be automatically installed - this can take a considerable amount of time for packages that need C or FORTRAN code compilation.

The following packages are needed to run this notebook:

- `reticulate` - R-Python interface, required to use `dxdata` package that connects to Spark database and allows retrieval of phenotypic data
- `dplyr` - tabular data manipulation in R, require to pre-process and filtering of phenotypic data
- `parallel` - parallel computation in R
- `bigsnpr` - run statistics on file-backed arrays, needed to calculate the approximate singular value decomposition (SVD) needed for PCA plots
- `bigparallelr` - controls parallel computation using file-backed arrays
- `ggplot2` - needed for graphics 
- `readr` - read and write tabular file formats: csv, tsv, tdf, etc.v, tdf, etc.
- `skimr` - provide summary statistics about variables in data frames, data tables and vectors


In [ ]:
if(!require(pacman)) install.packages("pacman")
pacman::p_load(dplyr, parallel, bigsnpr, ggplot2, readr, tidyr, bigparallelr, skimr)

## Download and read phenotypes

In this example, we will use the data table created in **Notebook 201** and *.bed files from **107**

In [2]:
system('dx download -r pheno/pheno_data_hypertension.csv', intern=TRUE)

character(0)

In [ ]:
pheno0 <- read_csv('pheno_data_hypertension.csv', show_col_types = FALSE)
pheno0[1:4,2:8]

## Read the EIDs of individuals in the exome cohort from `.fam` file

In [5]:
system('dx download -r bed_maf/maf_flt_8chroms*', intern=TRUE) #download .fam and associated files

character(0)

In [ ]:
exome_eids <- read_tsv('maf_flt_8chroms.fam', col_names = LETTERS[1:6], show_col_types = FALSE) %>% pull(B)

## Limit the phenotypes to the participants in the cohort 

In [22]:
pheno <- pheno0 %>% 
    filter(eid %in% exome_eids) %>%
    drop_na

## Filter phenotype records to hypertension positive ones and adds a 10k machine control sample of unaffected individuals

In [ ]:
smpl <- rbind(
    pheno %>% 
        filter(medical_records & self_reported & blood_pressure_cutoff),
    pheno %>%
        filter(!medical_records & !self_reported & !blood_pressure_cutoff) %>%
          sample_n(10000)
)
head(smpl[1:4,2:8])

## Read in the genotypes of individuals selected above from fam files

In [25]:
bedfile <- normalizePath("maf_flt_8chroms.bed")
tmpfile <- normalizePath("bigsnpr_hypertension_smpl", mustWork = FALSE)
if( length(dir(pattern=tmpfile)) ) unlink(dir(pattern=tmpfile))
snp_readBed2(bedfile, backingfile = tmpfile, ind.row=which(exome_eids %in% smpl$eid))

[1] "/opt/notebooks/bigsnpr_hypertension_smpl.rds"

## Preview genotypes

In [ ]:
genotypes_smpl <- snp_attach(paste0(tmpfile, ".rds"))
str(genotypes_smpl, max.level = 2, strict.width = "cut")

## Calculate singular value decomposition (SVD) and visualise it as a PCA plot

In [27]:
options(bigstatsr.check.parallel.blas = FALSE)
NCORES = round(nb_cores()/2)
assert_cores(NCORES)
message('INFO: Using ', NCORES, ' cores')

G   <- genotypes_smpl$genotypes
CHR <- genotypes_smpl$map$chromosome
POS <- genotypes_smpl$map$physical.pos
ind.excl <- snp_indLRLDR(infos.chr = as.integer(as.factor(CHR)), infos.pos = POS)
ind.keep <- snp_clumping(G, infos.chr = as.integer(as.factor(CHR)),exclude = ind.excl,ncores = NCORES)
obj.svd <- big_randomSVD(G, fun.scaling = snp_scaleBinom(), ind.col = ind.keep, ncores = NCORES)

INFO: Using 8 cores



## Prepare a PCA table

In [ ]:
pca <- cbind(genotypes_smpl$fam$sample.ID, obj.svd$u)
colnames(pca) <- c('eid', paste0('PC', 1:10))

pca <- pca %>% as_tibble %>% left_join(smpl, by = 'eid')
pca %>% skim %>% .[c(1:8, 11, 14)]

## Polygenic risk scores

### Divide the dataset into train set and test set

We divide the data into train and test sets in an 80:20 ratio:
- 15450 individuals are used to train the model 
- 3863 individuals are used to test the model

In [11]:
ind.train <- sample(nrow(G), nrow(G)*0.8)
ind.test <- setdiff(rows_along(G), ind.train)

length(ind.train)
length(ind.test)

[1] 29271

[1] 7318

### Fit the logistic regression model

We fit the lasso penalized regression model for a Filebacked Big Matrix. 
We use the first 10 principal components calculated from the variant matrix as covariates.

In [12]:
cmsa.logit <- big_spLogReg(
    X = G, 
    y01.train = as.numeric(pca$medical_records & pca$self_reported & pca$blood_pressure_cutoff)[ind.train], 
    ind.train = ind.train, 
    covar.train = obj.svd$u[ind.train, ],
    alphas = c(1, 0.5, 0.05, 0.001),
    ncores = NCORES)

## Calculate the area under the ROC curve

First, we predict the classes for our test dataset.

In [13]:
preds <- predict(
    cmsa.logit, 
    X = G, 
    ind.row = ind.test, 
    covar.row = obj.svd$u[ind.test, ])

Then we calculate the area under the receiver operating characteristic (ROC) curve.
The area under the ROC curve (AUC) is a good metric for classifier model performance. 
The perfect model would have an AUC of 1, while a random model would produce an AUC of 0.5.
In our case, the ~0.52 indicate a poor model performance. This is expected, since we used a very small subset of both individuals and variants in this example.

In [14]:
AUC(preds, as.numeric(pca$medical_records & pca$self_reported & pca$blood_pressure_cutoff)[ind.test])

[1] 0.5222176